In [4]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [5]:
import configparser

In [6]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10']

# Plate class

In [7]:
class plate():
    """
    Handles plates. Plate may be a real "plate", such as 96-well plate, or the other
    type of sample array. For example, 8-PCR tube stripe is also considered a plate.
    """
    
    #TODO: Sometimes plates are smaller than corresponding racks. Need capability to shift
    # plate related to rack
    
    def __init__(self, plate_name, plate_type):
        """
        Initializes plate of samples (array of samples)
        
        There should be a config files placed in configs/%plate_type%.ini and configs/%plate_type%_well.ini
        """
        
        self.plate_data = {}
        # Reading parameters for config file
        config = configparser.ConfigParser()
        config_path = 'configs/' + plate_type + '.ini'
        config.read(config_path)
        
        self.columns = int(config['wells']['columns'])
        self.rows = int(config['wells']['rows'])
        self.dist_between_cols = float(config['wells']['distance_between_columns'])
        self.dist_between_rows = float(config['wells']['distance_between_rows'])
        self.dist_cntr_to_1st_col = float(config['wells']['distance_center_to_1st_column'])
        self.dist_cntr_to_1st_row = float(config['wells']['distance_center_to_1st_row'])

        self.samples_list = self._initSamples(plate_name, plate_type)
        
    def _initSamples(self, plate_name, plate_type):
        samples_list = []
        for col in range(self.columns):
            for row in range(self.rows):
                s_name = plate_name + '_col_' + str(col) + '_row_' + str(row)
                s = samples.sample(sample_name=s_name, sample_type=plate_type+'_well')
                # Setting a position of a sample
                s.sample_data['x_well'] = col
                s.sample_data['y_well'] = row
                samples_list.append(s)
        return samples_list
    
    def getSample(self, column, row):
        for s in self.samples_list:
            if (s.sample_data['x_well'] == column) and (s.sample_data['y_well'] == row):
                return s
            
    def getSamples(self, col_row_list):
        """
        Returns list of samples objects
        Input:
            col_row_list
                List of sample coordinates in the plate. Each coordinate is 2 element list: [column, row]
                Example: [[3, 1], [6, 7], [11, 0]]
        Returns:
            List of sample objects
        """
        samples_to_return_list = []
        for col_row in col_row_list:
            col = col_row[0]
            row = col_row[1]
            s = self.getSample(col, row)
            samples_to_return_list.append(s)
        return samples_to_return_list
    
    def getAllSamples(self):
        return self.samples_list
    
    def place(self, rack):
        self.plate_data['rack'] = rack
        sample_list = self.getAllSamples()
        for s in sample_list:
            s.sample_data['rack'] = rack

In [8]:
# I have to create a new json file with all the parameters
# This is the relation of liquid volume and the distance from the liquid leve to the top of the tube
# The tube has no markings, so I have to fill the tube with liquid and manually measure its level
vol_vs_z = {
    0: 20.0,
    50: 13.77,
    80: 11.24,
    100: 10.30,
    150: 8.26,
    200: 5.88,
    250: 3.50,
    300: 1.40,
}

In [9]:
# Now this is the value on how much the tube extends above the rack
sample_top_dz = {
    "96wells_rack": 8.4,
}

In [10]:
# Info on how to grab the tube with a gripper
# This is a plate well, so gripper parameters are of entire plate.
# Those exact parameters may not be used at all.
# Those parameters need to be found experimentally as well
gripper_param = {
    "open_to_approach_uncapped": 80,
    "grip_uncapped": 76,
}

In [11]:
param = {
    'volume_vs_z': vol_vs_z,
    'sample_top_dz': sample_top_dz,
    'gripper_param': gripper_param,
        }

In [12]:
with open('configs/PCR_Plate_well.json', 'w') as fp:
    json.dump(param, fp)

# Testing plate class

In [83]:
p = plate('test_plate', 'PCR_Plate')

In [14]:
rack96well = racks.rack(rack_name='96well_rack', rack_type='96wells_rack')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [15]:
p.getSamples([[0, 0], [4, 7], [11, 7]])[0].sample_data

{'sample_name': 'test_plate_col_0_row_0',
 'sample_type': 'PCR_Plate_well',
 'x_well': 0,
 'y_well': 0}

In [84]:
p.place(rack96well)

In [17]:
p.getSamples([[0, 0], [4, 7], [11, 7]])[0].sample_data

{'sample_name': 'test_plate_col_0_row_0',
 'sample_type': 'PCR_Plate_well',
 'x_well': 0,
 'y_well': 0,
 'rack': <racks.rack at 0x2013fe31488>}

In [31]:
water = samples.sample(sample_name='water', sample_type='50mL_tube')
water.place(r50, 0, 0)
water.setVolume(17500)

In [19]:
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [20]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack', x_slot=5, y_slot=3)

# Pipetting

In [26]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [137]:
ar.home()

In [28]:
stp = tools.stationary_touch_probe(ar)

In [72]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [73]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [68]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)

In [22]:
samples_list = p.getAllSamples()

In [70]:
vol_list = []
for s in samples_list:
    vol_list.append(50)

In [71]:
samples_list[0].sample_data

{'sample_name': 'test_plate_col_0_row_0',
 'sample_type': 'PCR_Plate_well',
 'x_well': 0,
 'y_well': 0,
 'rack': <racks.rack at 0x1af82be75c8>}

In [72]:
samples_list[1].sample_data

{'sample_name': 'test_plate_col_0_row_1',
 'sample_type': 'PCR_Plate_well',
 'x_well': 0,
 'y_well': 1,
 'rack': <racks.rack at 0x1af82be75c8>}

In [65]:
p1000.getToSample(samples_list[2])

In [67]:
ar.move(z=350)

In [73]:
p1000.distributeLiquid(water, samples_list, vol_list, raise_z=320)

In [48]:
ar.move(z=0)

In [34]:
p1000.dropTipToWaste(waste_rack, raise_z=300)

In [110]:
p1000.returnTool()

# Gradient

In [23]:
def generateGradient(sample_list, axis, vol_list):
    plate_vol_list = []
    if axis == 'x':
        key = 'x_well'
    else:
        key = 'y_well'
    for s in sample_list:
        well = s.sample_data[key]
        plate_vol_list.append(vol_list[well])
    return plate_vol_list

In [92]:
def calcBuffer(max_vol, volumes_list):
    buffer_vol_list = []
    le = len(volumes_list[0])
    for i in range(le):
        vol_reagents = 0
        for reagent_vol_list in volumes_list:
            single_reagent_vol = reagent_vol_list[i]
            vol_reagents = vol_reagents + single_reagent_vol
        buffer_vol = max_vol - vol_reagents
        if buffer_vol < 0:
            buffer_vol = 0
        buffer_vol_list.append(buffer_vol)
    return buffer_vol_list

In [85]:
blue_gradient = generateGradient(samples_list, 'x', [0, 2, 5, 10, 20, 30, 40, 50, 75, 100, 125, 150])

In [87]:
red_gradient = generateGradient(samples_list, 'y', [0, 5, 10, 25, 50, 75, 100, 150])

In [93]:
buffer_vols = calcBuffer(300, [blue_gradient, red_gradient])

In [126]:
def pickSamplesForPipette(samples_list, vol_list):
    red_sample_list = []
    red_vol_list = []
    green_sample_list = []
    green_vol_list = []
    blue_sample_list = []
    blue_vol_list = []
    for vol, sample in zip(vol_list, samples_list):
        if vol > 0 and vol <= 20:
            red_sample_list.append(sample)
            red_vol_list.append(vol)
        elif vol > 20 and vol <= 200:
            green_sample_list.append(sample)
            green_vol_list.append(vol)
        elif vol > 200:
            blue_sample_list.append(sample)
            blue_vol_list.append(vol)
    return {'p20': (red_sample_list, red_vol_list),
            'p200': (green_sample_list, green_vol_list),
            'p1000': (blue_sample_list, blue_vol_list)}

In [129]:
buffer_vols_dict = pickSamplesForPipette(p.getAllSamples(), buffer_vols)
blue_vols_dict = pickSamplesForPipette(p.getAllSamples(), blue_gradient)
red_vols_dict = pickSamplesForPipette(p.getAllSamples(), red_gradient)

In [139]:
p = plate('gradient', 'PCR_Plate')
rack96well = racks.rack(rack_name='96well_rack', rack_type='96wells_rack')
p.place(rack96well)

In [136]:
buffer_vols_dict['p1000']

([<samples.sample at 0x2014015c708>,
 [300,
  295,
  290,
  275,
  250,
  225,
  298,
  293,
  288,
  273,
  248,
  223,
  295,
  290,
  285,
  270,
  245,
  220,
  290,
  285,
  280,
  265,
  240,
  215,
  280,
  275,
  270,
  255,
  230,
  205,
  270,
  265,
  260,
  245,
  220,
  260,
  255,
  250,
  235,
  210,
  250,
  245,
  240,
  225,
  225,
  220,
  215])

In [154]:
p200_tip_rack = racks.consumables(rack_name='p200_tips_1', rack_type='p200_tips')
p20_tip_rack = racks.consumables(rack_name='p20_tips_1', rack_type='p20_tips')

In [140]:
# Filling bigger volumes
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')
xt, yt, zt = calibration.calibrateTool(p1000, stp)

xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)
samples_list = buffer_vols_dict['p1000'][0]
volumes_list = buffer_vols_dict['p1000'][1]
p1000.distributeLiquid(yellow, samples_list, volumes_list, raise_z=320, touch_wall=True)
p1000.dropTipToWaste(waste_rack, raise_z=300)

p1000.returnTool()

In [141]:
# Filling medium volumes
p200 = tools.pipettor.getTool(robot=ar, tool_name='p200_tool')
xt, yt, zt = calibration.calibrateTool(p200, stp)

xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)
samples_list = buffer_vols_dict['p200'][0]
volumes_list = buffer_vols_dict['p200'][1]
p200.distributeLiquid(yellow, samples_list, volumes_list, raise_z=400, touch_wall=True)
p200.dropTipToWaste(waste_rack, raise_z=350)

xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)
samples_list = blue_vols_dict['p200'][0]
volumes_list = blue_vols_dict['p200'][1]
p200.distributeLiquid(blue, samples_list, volumes_list, raise_z=400, touch_wall=True)
p200.dropTipToWaste(waste_rack, raise_z=350)

xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=400)
samples_list = red_vols_dict['p200'][0]
volumes_list = red_vols_dict['p200'][1]
p200.distributeLiquid(red, samples_list, volumes_list, raise_z=400, touch_wall=True)
p200.dropTipToWaste(waste_rack, raise_z=350)

p200.returnTool()

In [155]:
# Filling smaller volumes
p20 = tools.pipettor.getTool(robot=ar, tool_name='p20_tool')
xt, yt, zt = calibration.calibrateTool(p20, stp)

xn, yn = p20_tip_rack.getNextConsumable()
p20.pickUpTip(p20_tip_rack, xn, yn, raise_z=400)
samples_list = blue_vols_dict['p20'][0]
volumes_list = blue_vols_dict['p20'][1]
p20.distributeLiquid(blue, samples_list, volumes_list, raise_z=400, touch_wall=True)
p20.dropTipToWaste(waste_rack, raise_z=350)

xn, yn = p20_tip_rack.getNextConsumable()
p20.pickUpTip(p20_tip_rack, xn, yn, raise_z=400)
samples_list = red_vols_dict['p20'][0]
volumes_list = red_vols_dict['p20'][1]
p20.distributeLiquid(red, samples_list, volumes_list, raise_z=400, touch_wall=True)
p20.dropTipToWaste(waste_rack, raise_z=350)

p20.returnTool()

In [149]:
p20.dropTipToWaste(waste_rack, raise_z=350)

In [150]:
p20.returnTool()

In [119]:
p200.dropTipToWaste(waste_rack, raise_z=300)

In [120]:
p200.returnTool()

In [156]:
ar.home()

In [32]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)

In [33]:
p1000.distributeLiquid(water, samples_list, vol_list, raise_z=320)

In [57]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')

In [58]:
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
[x[0].place(ep1, x[1], 3) for x in zip(sample_list, range(5))]

[None, None, None, None, None]

In [59]:
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=350)

In [60]:
ethanol_vol_list = [200 for x in sample_list]

In [61]:
p1000.distributeLiquid(water, sample_list, ethanol_vol_list, raise_z=350)

# Michigan

In [138]:
yellow = samples.sample(sample_name='yellow', sample_type='50mL_tube')
yellow.place(r50, 0, 0)
yellow.setVolume(50000)

In [36]:
red = samples.sample(sample_name='red', sample_type='50mL_tube')
red.place(r50, 1, 0)
red.setVolume(50000)

In [37]:
blue = samples.sample(sample_name='blue', sample_type='50mL_tube')
blue.place(r50, 2, 0)
blue.setVolume(50000)

In [60]:
p = plate('michigan', 'PCR_Plate')
rack96well = racks.rack(rack_name='96well_rack', rack_type='96wells_rack')
p.place(rack96well)

In [61]:
blue_coord_truth_table = [
    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
]

In [65]:
blue_sample_list = []
blue_vol_list = []
yellow_sample_list = []
yellow_vol_list = []
row_count = 0
col_count = 0
for i in blue_coord_truth_table:
    for j in i:
        s = p.getSample(column=col_count, row=row_count)
        if j == 1:
            blue_sample_list.append(s)
            blue_vol_list.append(200)
        else:
            yellow_sample_list.append(s)
            yellow_vol_list.append(200)
        col_count += 1
    row_count += 1
    col_count = 0

In [66]:
len(blue_sample_list)

56

In [67]:
len(yellow_sample_list)

40

In [69]:
len(blue_vol_list)

56

In [70]:
len(yellow_vol_list)

40

In [75]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)

In [76]:
p1000.distributeLiquid(yellow, yellow_sample_list, yellow_vol_list, raise_z=320, touch_wall=True)

In [77]:
ar.move(z=300)

In [81]:
p1000.dropTipToWaste(waste_rack, raise_z=300)

In [79]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)

In [80]:
p1000.distributeLiquid(blue, blue_sample_list, blue_vol_list, raise_z=320, touch_wall=True)